#### **Merging and Joining**
1. **Inner Join on Chinook Database**
   - Load the `chinook.db` database.
   - Perform an inner join between the `customers` and `invoices` tables on the `CustomerId` column.
   - Find the total number of invoices for each customer.


In [1]:
import sqlite3
import pandas as pd
with sqlite3.connect('data/chinook.db') as connection:
    customers = pd.read_sql("select * from customers", connection)
    invoices = pd.read_sql("select * from invoices", connection)
    print(customers.merge(invoices, on='CustomerId'))


     CustomerId FirstName    LastName  \
0             1      Luís   Gonçalves   
1             1      Luís   Gonçalves   
2             1      Luís   Gonçalves   
3             1      Luís   Gonçalves   
4             1      Luís   Gonçalves   
..          ...       ...         ...   
407          59      Puja  Srivastava   
408          59      Puja  Srivastava   
409          59      Puja  Srivastava   
410          59      Puja  Srivastava   
411          59      Puja  Srivastava   

                                              Company  \
0    Embraer - Empresa Brasileira de Aeronáutica S.A.   
1    Embraer - Empresa Brasileira de Aeronáutica S.A.   
2    Embraer - Empresa Brasileira de Aeronáutica S.A.   
3    Embraer - Empresa Brasileira de Aeronáutica S.A.   
4    Embraer - Empresa Brasileira de Aeronáutica S.A.   
..                                                ...   
407                                              None   
408                                              No

2. **Outer Join on Movie Data**
   - Load the `movie.csv` file.
   - Create two smaller DataFrames:
     - One with only `director_name` and `color`.
     - Another with `director_name` and `num_critic_for_reviews`.
   - Perform a left join and then a full outer join on `director_name`.
   - Count how many rows are in the resulting DataFrames for each join type.


In [2]:
movie_db = pd.read_csv('data/movie.csv')
color = movie_db[['director_name', 'color']]
reviews = movie_db[['director_name', 'num_critic_for_reviews']]
left_join = color.merge(reviews, on='director_name', how='left')
print(left_join)
print(len(left_join))
full_outer_join = color.merge(reviews, on='director_name', how='outer')
print(full_outer_join)
print(len(full_outer_join))

          director_name  color  num_critic_for_reviews
0         James Cameron  Color                   723.0
1         James Cameron  Color                   315.0
2         James Cameron  Color                   210.0
3         James Cameron  Color                    94.0
4         James Cameron  Color                    82.0
...                 ...    ...                     ...
30295  Benjamin Roberds  Color                    13.0
30296       Daniel Hsia  Color                    14.0
30297          Jon Gunn  Color                    11.0
30298          Jon Gunn  Color                    15.0
30299          Jon Gunn  Color                    43.0

[30300 rows x 3 columns]
30300
         director_name  color  num_critic_for_reviews
0        A. Raven Cruz  Color                     3.0
1           Aaron Hann  Color                    29.0
2      Aaron Schneider  Color                   160.0
3        Aaron Seltzer  Color                    99.0
4         Abel Ferrara  Color         

#### **Grouping and Aggregating**
1. **Grouped Aggregations on Titanic**
   - Group passengers by `Pclass` and calculate the following:
     - Average age.
     - Total fare.
     - Count of passengers.
   - Save the results to a new DataFrame.


In [3]:
df_titanic = pd.read_excel('data/titanic.xlsx')
grouped_df = df_titanic.groupby('Pclass').agg(
    average_age=('Age', 'mean'),
    total_fare=('Fare', 'sum'),
    passenger_count=('PassengerId', 'count')
).reset_index()
print(grouped_df)

   Pclass  average_age  total_fare  passenger_count
0       1    38.233441  18177.4125              216
1       2    29.877630   3801.8417              184
2       3    25.140620   6714.6951              491


2. **Multi-level Grouping on Movie Data**
   - Group the movies by `color` and `director_name`.
   - Find:
     - Total `num_critic_for_reviews` for each group.
     - Average `duration` for each group.


In [4]:
grouped_movie = movie_db.groupby(['color', 'director_name']).agg(
    total_num_critic_for_reviews = ('num_critic_for_reviews', 'count'),
    average_duration = ('duration', 'mean')
).reset_index()
print(grouped_movie)

                color       director_name  total_num_critic_for_reviews  \
0     Black and White      Akira Kurosawa                             1   
1     Black and White      Aleksey German                             1   
2     Black and White        Alex Garland                             1   
3     Black and White     Alexander Payne                             1   
4     Black and White    Alfred Hitchcock                             2   
...               ...                 ...                           ...   
2485            Color       Zoran Lisinac                             1   
2486            Color  Álex de la Iglesia                             1   
2487            Color    Émile Gaudreault                             1   
2488            Color        Éric Tessier                             1   
2489            Color       Étienne Faure                             1   

      average_duration  
0                202.0  
1                177.0  
2                108.0  

3. **Nested Grouping on Flights**
   - Group flights by `Year` and `Month` and calculate:
     - Total number of flights.
     - Average arrival delay (`ArrDelay`).
     - Maximum departure delay (`DepDelay`).


In [20]:
flights = pd.read_parquet('data/flights')
grouped_flights = flights.groupby(['Year', 'Month']).agg(
    total_num_flights=('Year', 'count'),
    average_arr_delay = ('ArrDelay', 'mean'),
    max_dep_delay=('DepDelay', 'max')

).reset_index()
print(grouped_flights)

MemoryError: Unable to allocate 5.46 GiB for an array with shape (109, 6729125) and data type object

---

#### **Applying Functions**
1. **Apply a Custom Function on Titanic**
   - Write a function to classify passengers as `Child` (age < 18) or `Adult`.
   - Use `apply` to create a new column, `Age_Group`, with these values.


In [5]:
def age_group(x):
    if x < 18:
        return 'Child'
    return 'Adult'
df_titanic['Age group'] = df_titanic['Age'].apply(age_group)
print(df_titanic)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

2. **Normalize Employee Salaries**
   - Load the `employee.csv` file.
   - Normalize the salaries within each department.


In [6]:
employees = pd.read_csv('data/employee.csv')
employees

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18
2,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03
3,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22
...,...,...,...,...,...,...,...,...,...,...
1995,1995,POLICE OFFICER,Houston Police Department-HPD,43443.0,White,Full Time,Male,Active,2014-06-09,2015-06-09
1996,1996,COMMUNICATIONS CAPTAIN,Houston Fire Department (HFD),66523.0,Black or African American,Full Time,Male,Active,2003-09-02,2013-10-06
1997,1997,POLICE OFFICER,Houston Police Department-HPD,43443.0,White,Full Time,Male,Active,2014-10-13,2015-10-13
1998,1998,POLICE OFFICER,Houston Police Department-HPD,55461.0,Asian/Pacific Islander,Full Time,Male,Active,2009-01-20,2011-07-02


In [12]:
employees['normalized_salary'] = employees.groupby('DEPARTMENT')['BASE_SALARY'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)
print(employees)

      UNIQUE_ID               POSITION_TITLE                     DEPARTMENT  \
0             0  ASSISTANT DIRECTOR (EX LVL)    Municipal Courts Department   
1             1            LIBRARY ASSISTANT                        Library   
2             2               POLICE OFFICER  Houston Police Department-HPD   
3             3            ENGINEER/OPERATOR  Houston Fire Department (HFD)   
4             4                  ELECTRICIAN    General Services Department   
...         ...                          ...                            ...   
1995       1995               POLICE OFFICER  Houston Police Department-HPD   
1996       1996       COMMUNICATIONS CAPTAIN  Houston Fire Department (HFD)   
1997       1997               POLICE OFFICER  Houston Police Department-HPD   
1998       1998               POLICE OFFICER  Houston Police Department-HPD   
1999       1999                 FIRE FIGHTER  Houston Fire Department (HFD)   

      BASE_SALARY                       RACE EMPLOY

3. **Custom Function on Movies**
   - Write a function that returns `Short`, `Medium`, or `Long` based on the duration of a movie:
     - `Short`: Less than 60 minutes.
     - `Medium`: Between 60 and 120 minutes.
     - `Long`: More than 120 minutes.
   - Apply this function to classify movies in the `movie.csv` dataset.


In [14]:
def classify_movie_duration(x):
    """classifies movie according to its duration: Short, Medium, Long"""
    if x < 60:
        return 'Short'
    elif x <= 120:
        return 'Medium'
    return 'Long'

movie_db['classified_duration'] = movie_db['duration'].apply(classify_movie_duration)
print(movie_db)


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,classified_duration
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,Long
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0,Long
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000,Long
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000,Long
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0,Long


---

#### **Using `pipe`**
1. **Pipeline on Titanic**
   - Create a pipeline to:
     - Filter passengers who survived (`Survived == 1`).
     - Fill missing `Age` values with the mean.
     - Create a new column, `Fare_Per_Age`, by dividing `Fare` by `Age`.


In [19]:
def filter_survivors(df):
    """filters the survived passengers"""
    return df[df['Survived'] == 1]

def fill_missing_age(df):
    return df.assign(Age=df['Age'].fillna(df['Age'].mean()))
def fare_per_age(df):
    df['Fare_Per_Age'] = df['Fare'] / df['Age']
    return df
# Apply the pipeline
result_df = (
    df_titanic
    .pipe(filter_survivors)
    .pipe(fill_missing_age)
    .pipe(fare_per_age)
)
print(result_df)

     PassengerId  Survived  Pclass  \
1              2         1       1   
2              3         1       3   
3              4         1       1   
8              9         1       3   
9             10         1       2   
..           ...       ...     ...   
875          876         1       3   
879          880         1       1   
880          881         1       2   
887          888         1       1   
889          890         1       1   

                                                  Name     Sex   Age  SibSp  \
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
8    Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)  female  27.0      0   
9                  Nasser, Mrs. Nicholas (Adele Achem)  female  14.0      1   
..                                                 ...     ...   ... 

2. **Pipeline on Flights**
   - Create a pipeline to:
     - Filter flights with a departure delay greater than 30 minutes.
     - Add a column `Delay_Per_Hour` by dividing the delay by the scheduled flight duration.


In [ ]:
def filter_long_delays(df):
    """filters long delays"""
    return df[df['DepDelay'] > 30]


def delay_per_hour(df):
    """adda a new column delay per hour"""
    df = df.copy()
    df['Delay_Per_Hour'] = df['DepDelay'] / df['Duration']
    return df


long_delay_flights = (
    flights
    .pipe(filter_long_delays)
    .pipe(delay_per_hour)
)

print(long_delay_flights.head())